In [1]:
import pandas as pd
import numpy as np

In [ ]:
Задание 3

In [68]:
# Выгрузите организации, где число сотрудников в 2020 году 15 и больше. 
# Если за 2020 год данных по сотрудникам нет, то возьмите за 2019 год. 
# Отсортируйте по возрастанию числа сотрудников. Если число сотрудников 
# одинаковое, то организации должны идти по убыванию ID.

In [378]:
# создаем подключение к базе данных:

import sqlite3

con = sqlite3.connect('indicators_db.sqlite3')

con.execute('select * from org_indicators_2020_2022')

# обращаемся к объекту подключения и прописываем запрос
# присваеваем резултат переменной 2020_2022

cursor = con.execute('select * from org_indicators_2020_2022')

# обращаемся к объекту подключения и прописываем запрос
# присваеваем резултат переменной 2019

cursor1 = con.execute('select * from org_indicators_2019')

# вытягиваем наши данные из объекта курсора <sqlite3.Cursor at 0x16ad20d32c0>

row = cursor.fetchall()

row1 = cursor1.fetchall()

df_all_indicators = pd.DataFrame(row, columns = ['ID', 'Year', 'Indicator', 'Value'])

df_all_indicators1 = pd.DataFrame(row1, columns = ['ID', 'Year', 'Indicator', 'Value'])

In [379]:
df = df_all_indicators[(df_all_indicators['Indicator'] == 'Число сотрудников')]

In [380]:
df_pivot = df.pivot(values = 'Value', columns = 'Year', index = 'ID')

In [381]:
df_pivot = df_pivot.reindex([2020], axis = 1)

In [382]:
df_pivot

Year,2020
ID,
19630,NaN
19643,17.0
19647,16.0
19657,NaN
19677,17.0
...,...
375884,8.0
375933,NaN
375954,16.0


In [383]:
df_2019 = df_all_indicators1[(df_all_indicators1['Indicator'] == 'Число сотрудников')]

In [384]:
df_pivot_2019 = df_2019.pivot(values = 'Value', columns = 'Year', index = 'ID')

In [385]:
df_pivot_2019

Year,2019
ID,
19630,7
19647,14
19681,18
19688,18
19708,6
...,...
375783,6
375834,16
375884,7


In [386]:
df = pd.merge(df_pivot, df_pivot_2019, how = 'outer', left_index = True, right_index = True)

In [388]:
df = df.reindex([2019, 2020], axis = 1)

In [390]:
df[2020] = df[2020].combine_first(df[2019])

In [391]:
df[df[2020] >= 15]

Year,2019,2020
ID,,
19643,NaN,17.0
19647,14.0,16.0
19677,NaN,17.0
19681,18.0,16.0
19688,18.0,16.0
...,...,...
375648,NaN,15.0
375654,14.0,18.0
375834,16.0,19.0


In [392]:
df = df[df[2020] >= 15]

In [393]:
df = df.reset_index()

In [394]:
df = df.reindex(['ID', 2020], axis = 1)

In [400]:
df

Year,ID,2020
0,19643,17
1,19647,16
2,19677,17
3,19681,16
4,19688,16
...,...,...
3136,375648,15
3137,375654,18
3138,375834,19
3139,375954,16


In [399]:
df[2020] = df[2020].astype('int64')

In [401]:
df.sort_values(by = [2020, 'ID'], ascending = [True, False])

Year,ID,2020
3136,375648,15
3133,375575,15
3131,375534,15
3126,375320,15
3123,375119,15
...,...,...
41,20838,19
30,20503,19
25,20315,19
18,20045,19


In [402]:
df = df.sort_values(by = [2020, 'ID'], ascending = [True, False])

In [403]:
df.columns = ['Идентификатор', 'Сотрудники 2019_2020']

In [404]:
df

,Идентификатор,Сотрудники 2019_2020
3136,375648,15
3133,375575,15
3131,375534,15
3126,375320,15
3123,375119,15
...,...,...
41,20838,19
30,20503,19
25,20315,19
18,20045,19


In [405]:
df['Идентификатор'].sum()

747477261

In [406]:
df.to_csv('011_pivot_task_3.csv', index = False)